In [ ]:
! pip install -q kaggle #installing kaggle

In [ ]:
from google.colab import files
files.upload()                        # connecting kaggle with google colab via jason file

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"saadahmad127","key":"5d229b238808113c0cddf9a67ceddbdd"}'}

In [ ]:
! mkdir ~/.kaggle # kagle integration

In [ ]:
! cp kaggle.json ~/.kaggle/ # kagle json integration

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json # kagle integration

In [ ]:
! kaggle datasets download -d volodymyrgavrysh/pneumothorax-binary-classification-task -p /content # downloadind dataset from kaggle

100% 774M/774M [00:09<00:00, 80.4MB/s]
100% 774M/774M [00:09<00:00, 89.3MB/s]


In [ ]:
!unzip '/content/pneumothorax-binary-classification-task.zip'

# Data Processing

# New Section

In [ ]:
#Data Processing

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/train_data.csv')

In [ ]:
df

In [ ]:
df['target'].value_counts()

1    1597
0     430
Name: target, dtype: int64

Index(['Unnamed: 0.1', 'Unnamed: 0', 'file_name', 'target'], dtype='object')

In [ ]:
# importing necessary Libraries
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from skimage import exposure
from skimage.feature import hog

In [ ]:
!mkdir '/content/separated' 

In [ ]:
import pandas as pd
import shutil
import os

# Read the CSV file with labels
labels_df = pd.read_csv('/content/train_data.csv')

# Create the output directories for each class
output_dir_normal = '/content/separated/normal_images/'
output_dir_pneumothorax = '/content/separated/pneumothorax_images/'
os.makedirs(output_dir_normal, exist_ok=True)
os.makedirs(output_dir_pneumothorax, exist_ok=True)

# Loop through each row in the labels dataframe
for i in range(len(labels_df)):
    # Get the filename and class label
    filename ='/content/small_train_data_set/small_train_data_set/' + labels_df['file_name'][i]

    class_label = labels_df['target'][i]
    
    # Create the output path based on the class label
    if class_label == 0:
        output_path = os.path.join(output_dir_normal, labels_df['file_name'][i])
    else:
        output_path = os.path.join(output_dir_pneumothorax, labels_df['file_name'][i])
    print(filename, output_path)
    # Copy the image file to the output directory
    shutil.copy(filename, output_path)


In [ ]:
import os
import numpy as np
import cv2
from keras.preprocessing.image import ImageDataGenerator


In [ ]:
data_gen = ImageDataGenerator(rotation_range=20,
                              width_shift_range=0.1,
                              height_shift_range=0.1,
                              shear_range=0.1,
                              zoom_range=0.1,
                              horizontal_flip=True,
                              vertical_flip=True,
                              fill_mode='nearest')


In [ ]:
!mkdir '/content/normal'
!mkdir '/content/normal/normal'


In [ ]:
!cp -r /content/separated/normal_images/* '/content/normal/normal'  #copying images


In [ ]:
data_path = '/content/normal' # path of normal images


In [ ]:
batch_size = 32  # making bathc sizes for datset
target_size = (256, 256)

data_generator = data_gen.flow_from_directory(data_path,
                                               target_size=target_size,
                                               batch_size=batch_size
                                               )


Found 3261 images belonging to 2 classes.


In [ ]:
!mkdir '/content/augmented'  #agumentation of dataset

In [ ]:
save_path = '/content/augmented/'


In [ ]:
import math

In [ ]:
math.ceil(2.1)

3

In [ ]:
n_augmented_images = 4 #separating normal and dieseas images
augmented=[]
augmented_labels=[]
for i in range(n_augmented_images):
    data_generator = data_gen.flow_from_directory(data_path,
                                               target_size=target_size,
                                               batch_size=batch_size)
    for k in range(math.floor(430/batch_size)):                                       
      batch = data_generator.next()
      images = batch[0]
      labels=batch[1]

      for j in range(batch_size):
          image = images[j]
          
          filename = 'image_{}_{}_{}.png'.format(i,k, j)
          file_path = os.path.join(save_path, filename)
          augmented.append(file_path)
          augmented_labels.append(labels[j])

          cv2.imwrite(file_path, image)


Found 430 images belonging to 1 classes.
Found 430 images belonging to 1 classes.
Found 430 images belonging to 1 classes.
Found 430 images belonging to 1 classes.


In [ ]:
len(augmented_labels)

1664

In [ ]:
!mkdir '/content/final_augmented'
!mkdir '/content/final_augmented/normal'
!mkdir '/content/final_augmented/disease'



In [ ]:
!cp -r /content/augmented/* '/content/final_augmented/normal'


In [ ]:
!cp -r /content/separated/pneumothorax_images/* '/content/final_augmented/disease'


In [ ]:
data_path = '/content/final_augmented'

In [ ]:
data_gen = ImageDataGenerator()
batch_size = 32
target_size = (256, 256)

data_generator = data_gen.flow_from_directory(data_path,
                                               target_size=target_size,
                                               batch_size=batch_size
                                               )

Found 3261 images belonging to 2 classes.


In [ ]:
3261/32

101.90625

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from skimage import exposure
from skimage.feature import hog

In [ ]:
features=[]
labels=[]
for i in range(math.floor(3261/batch_size)):                                       
  batch = data_generator.next()
  images = batch[0]
  labels_batch=batch[1]
  labels.extend(labels_batch)
  for j in range(batch_size):
      image = images[j]
      gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
      # Apply histogram equalization
      eq_image = exposure.equalize_hist(gray)
      
      # Apply image resizing or cropping
      resized_image = cv2.resize(eq_image, (256, 256))
      hog_features = hog(resized_image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(3, 3), block_norm='L2-Hys')
    
    # Append the HOG features and its label to the features and labels lists
      features.append(hog_features)
      


In [ ]:
data_generator.class_indices

{'disease': 0, 'normal': 1}

In [ ]:
lab=[]
for i in labels:
  if (i==[0,1]).all():
    lab.append(0)
  else:
    lab.append(1)
    

In [ ]:
pd.Series(lab).value_counts()

0    1646
1    1583
dtype: int64

In [ ]:
# Split the data and labels into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, lab, test_size=0.2, random_state=42)

from sklearn.svm import SVC

# Initialize the SVM classifier
clf = SVC(kernel='linear', probability=True, random_state=42)

# Train the classifier on the training data and labels
clf.fit(X_train, y_train)

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Make predictions on the testing data
y_pred = clf.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))

# Generate a confusion matrix and classification report
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

class_report = classification_report(y_test, y_pred)
print("Classification Report:\n", class_report)


Accuracy: 98.45%
Confusion Matrix:
 [[329   5]
 [  5 307]]
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       334
           1       0.98      0.98      0.98       312

    accuracy                           0.98       646
   macro avg       0.98      0.98      0.98       646
weighted avg       0.98      0.98      0.98       646



In [ ]:
import pickle
s = pickle.dumps(clf)
clf2 = pickle.loads(s)
clf2.predict(X_train[0:1])

array([1])

In [ ]:
pickle.dump(clf, open('model.pkl', 'wb'))

In [ ]:
# pickled_model = pickle.load(open('model.pkl', 'rb'))
# pickled_model.predict(X_test)


In [ ]:
from google.colab import drive

In [ ]:
drive.mount('gdrive')

Mounted at gdrive


In [ ]:
! ls '/content/gdrive/MyDrive'

In [ ]:
pickle.dump(clf, open('/content/gdrive/MyDrive/model.pkl', 'wb'))